In [225]:
!pip install numpy

In [226]:
!pip install openai singlestoredb pandas langchain==0.1.8 langchain-community==0.0.21 langchain-core==0.1.25

In [227]:
!pip install langchain-openai==0.0.6

## vector embedding

In [228]:
def word_to_vector(word):
    # define basic rules for our vector embedding by initialising a vector of 5 dimensions
    vector  = [0]*5

    #Rule1: length of word normalized to 10 chars
    vector[0] = len(word)/10
    #Rule 2: Number of vowels in the word (normalized to the length of the word)
    vowels = 'aeiou'
    vector[1] = sum(1 for char in word if char in vowels)/len(word)
    # Rule 3: Whether the word starts with a vowel (1) or not (0)
    vector[2] = 1 if word[0] in vowels else 0
    # Rule 4: Whether the word ends with a vowel (1) or not (0)
    vector[3]  = 1 if word[-1] in vowels else 0
    # Rule 5: Percentage of consonants in the word
    vector[4] = sum(1 for char in word if char not in vowels and char.isalpha()) / len(word)

    return vector


In [229]:
#example usage
word = "example"
print(word_to_vector(word))

[0.7, 0.42857142857142855, 1, 1, 0.5714285714285714]


## vector similarity example

In [230]:
import numpy as np

def cosine_similarity(vector_a,vector_b):
    #calculate the dot product of vectors
    dot_product = np.dot(vector_a,vector_b)
    #calculate the norm(magnitude) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    #find the cosine similarity
    similarity = dot_product/(norm_a*norm_b)
    return similarity
    


In [231]:
#example usage
word1 = "example"
word2 = "sample"
vector_a = word_to_vector(word1)
vector_b = word_to_vector(word2)
simi = cosine_similarity(vector_a,vector_b)
print(simi)

      

0.8108320947993962


## embedding models

In [237]:
OPENAI_KEY = "sk-proj-H7TbOTxeGDKJiLRnwsoeT3BlbkFJjSZA5jQazwhwpBeczhgx"
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

def openAIEmbeddings(input):
 response = client.embeddings.create(
     input="input",
     model="text-embedding-3-small"
 )
 return response.data[0].embedding


In [238]:
print(openAIEmbeddings("Golden Retreiver"))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

##Creating a vector database with SingleStoreDB

## Add OpenAI API key as an environment variable.

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-H7TbOTxeGDKJiLRnwsoeT3BlbkFJjSZA5jQazwhwpBeczhgx'

## Next, import the libraries, mention the file you want to use in the example, load the file, split it and add the file content into SingleStore database. Finally, ask the query related to the document you used.

In [313]:
import openai
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings`
from langchain_community.vectorstores.singlestoredb import SingleStoreDB
import os
import pandas as pd
import requests 

# URL of the public .txt file you want to use
file_url = "https://sherlock-holm.es/stories/plain-text/stud.txt"
# Send a GET request to the file URL
response = requests.get(file_url)

SyntaxError: invalid syntax (1303279027.py, line 4)

In [318]:
# Proceed if the file was successfully downloaded
if response.status_code==200:
     file_content = response.text
     # Save the content to a file
     file_path = 'donwloaded_file.txt'
     with open(file_path, 'w', encoding='utf-8') as f:
        f.write(file_content)

     # Now, you can proceed with your original code using 'downloaded_example.txt'
     # Load and process documents\
     loader = TextLoader(file_path)  # Use the downloaded document
     documents = loader.load()
     text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
     docs = text_splitter.split_documents(documents)
     # Generate embeddings and create a document search database
     OPENAI_KEY = "sk-proj-H7TbOTxeGDKJiLRnwsoeT3BlbkFJjSZA5jQazwhwpBeczhgx"  # Replace with your OpenAI API key
     embeddings = OpenAIEmbeddings(api_key=OPENAI_KEY)
     # Create Vector Database
     vector_database = SingleStoreDB.from_documents(docs, embeddings, table_name="scarlet")  # Replace "your_table_name" with your actual table name
     query = "which university did he study?"
     docs = vector_database.similarity_search(query)
     print(docs[0].page_content)
else:
     print("Failed to download the file. Please check the URL and try again.")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Once run the above code, you will see a tab to enter your query/question you would like to ask about the sherlock holmes story we have referenced.


We retrieved the relevant information from the provided data and then used this information to guide the response generation process. By converting our file into embeddings and storing them in the SingleStore database, we created a retrievable corpus of information. This way, we ensured that the responses are not only relevant but also rich in content derived from the provided dataset.